In [1]:
!pip -q install --upgrade aperturedb pandas python-dateutil tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.8/137.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 81.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that

In [11]:
import uuid
import re
import pandas as pd
from dateutil import parser as dtparser
from pathlib import Path
from tqdm.auto import tqdm
from google.colab import userdata

from aperturedb.CommonLibrary import create_connector
from aperturedb.ParallelLoader import ParallelLoader

In [6]:
# ------------ CONFIG ------------
APERTUREDB_KEY = userdata.get('APERTUREDB_KEY')
CSV_PATH = "/content/mlops-events-enriched.csv"

ENTITY_CLASS = "Talk"  # entity class name in ApertureDB
BATCHSIZE = 50
NUMTHREADS = 8
SHOW_STATS = True

In [18]:
# ------------ HELPERS ------------

def clean_str_or_none(v):
    import pandas as pd
    # If a Series sneaks in, take the first non-null scalar
    if isinstance(v, pd.Series):
        v = v.dropna()
        v = v.iloc[0] if len(v) else None
    if v is None or (isinstance(v, float) and pd.isna(v)):
        return None
    s = str(v).strip()
    return s if s else None

def coerce_int_or_none(v):
    import pandas as pd, re
    if isinstance(v, pd.Series):
        v = v.dropna()
        v = v.iloc[0] if len(v) else None
    if v is None or (isinstance(v, float) and pd.isna(v)):
        return None
    s = str(v)
    m = re.search(r"\d+", s.replace(",", ""))
    return int(m.group(0)) if m else None

def to_aperturedb_date(value):
    # Accepts things like "Dec 5, 2024" and returns "2024-12-05"
    import pandas as pd
    from dateutil import parser as dtparser
    if isinstance(value, pd.Series):
        value = value.dropna()
        value = value.iloc[0] if len(value) else None
    if value is None or (isinstance(value, float) and pd.isna(value)):
        return None
    try:
        dt = dtparser.parse(str(value))
        return dt.strftime("%Y-%m-%d")  # <-- date only (no time, no T/Z)
    except Exception:
        return None

def extract_youtube_id(url: str):
    """Try to get YouTube ID from either 'YouTube ID' column or 'YouTube Link' values."""
    if pd.isna(url):
        return None
    s = str(url).strip()
    # common forms
    m = re.search(r"[?&]v=([A-Za-z0-9_-]{6,})", s)
    if m:
        return m.group(1)
    m = re.search(r"youtu\.be/([A-Za-z0-9_-]{6,})", s)
    if m:
        return m.group(1)
    return None

def make_talk_id(talk_title: str, youtube_id: str):
    """
    Deterministic UUID: reruns won't generate a new ID.
    Uses UUID5 over a stable string (title + youtube_id).
    """
    base = f"{(talk_title or '').strip()}|{(youtube_id or '').strip()}"
    return str(uuid.uuid5(uuid.NAMESPACE_URL, base))


In [8]:
# ------------ LOAD DATA ------------
df = pd.read_csv(CSV_PATH)

# Keep only rows with a talk title
df = df[df["Talk Title"].notna()].copy().reset_index(drop=True)

In [19]:
class TalksGenerator:
    """
    ParallelLoader calls generator[start:stop] and expects
    a *list* of (commands, blobs) pairs for slices.
    For single int indices, a single (commands, blobs) pair is OK.
    """

    def __init__(self, dataframe: pd.DataFrame):
        # optional: remove duplicate column names defensively
        self.df = dataframe.loc[:, ~dataframe.columns.duplicated()].copy()

    def __len__(self):
        return len(self.df)

    def _row_to_cmd_blob(self, idx: int):
        def get(col):
            return self.df.at[idx, col] if col in self.df.columns else None

        talk_title   = clean_str_or_none(get("Talk Title"))
        youtube_id_c = clean_str_or_none(get("YouTube ID"))
        youtube_url  = clean_str_or_none(get("YouTube Link"))
        youtube_id   = youtube_id_c or extract_youtube_id(youtube_url)

        talk_id = make_talk_id(talk_title or "", youtube_id or "")

        props = {
            "talk_id": talk_id,
            "talk_title": talk_title,
        }
        if youtube_id:  props["youtube_id"]  = youtube_id
        if youtube_url: props["youtube_url"] = youtube_url

        kw_csv = clean_str_or_none(get("Top 3 keywords (in order)"))
        if kw_csv is not None:
            props["keywords_csv"] = kw_csv

        mapping = [
            ("Full Name", "speaker_name"),
            ("Company Name", "company_name"),
            ("Job Title", "job_title"),
            ("Abstract", "abstract"),
            ("What You'll Learn", "what_youll_learn"),
            ("Prerequiste Knowledge (if required)", "prereq_knowledge"),
            ("Track", "track"),
            ("Category 1", "category_primary"),
            ("Bio", "bio"),
            ("Relevant Industries", "industries"),
            ("What is Unique about your session", "unique_session_note"),
            ("Event", "event_name"),
        ]
        for csv_col, db_key in mapping:
            val = clean_str_or_none(get(csv_col))
            if val is not None:
                props[db_key] = val

        tech_level = coerce_int_or_none(get("Technical Level (1-7)"))
        if tech_level is not None:
            props["tech_level"] = tech_level

        views = coerce_int_or_none(get("yt_views"))
        if views is not None:
            props["yt_views"] = views

        iso_date = to_aperturedb_date(get("yt_published_date"))
        if iso_date is not None:
            props["yt_published_at"] = {"_date": iso_date}

        cmd = {
            "AddEntity": {
                "class": ENTITY_CLASS,
                "if_not_found": { "talk_id": ["==", talk_id] },
                "properties": props
            }
        }
        return [cmd], []  # (commands, blobs)

    def __getitem__(self, idx_or_slice):
        # Single index → one (commands, blobs) pair
        if isinstance(idx_or_slice, int):
            return self._row_to_cmd_blob(idx_or_slice)

        # Slice → list of (commands, blobs) pairs
        if isinstance(idx_or_slice, slice):
            start = idx_or_slice.start or 0
            stop  = idx_or_slice.stop  or len(self.df)
            step  = idx_or_slice.step  or 1
            out = []
            for i in range(start, min(stop, len(self.df)), step):
                out.append(self._row_to_cmd_blob(i))
            return out

        # Iterable of indices → list of pairs
        try:
            indices = list(idx_or_slice)
            out = []
            for i in indices:
                out.append(self._row_to_cmd_blob(int(i)))
            return out
        except Exception:
            # Fallback: treat as single index
            return self._row_to_cmd_blob(int(idx_or_slice))

    def get_indices(self):
        # Let ParallelLoader.query_setup() auto-create this index
        return { "entity": { ENTITY_CLASS: ["talk_id"] } }

In [20]:
# ------------ CONNECT & INGEST ------------
client = create_connector(key=APERTUREDB_KEY)
loader = ParallelLoader(client)

gen = TalksGenerator(df)

# auto-create indices based on get_indices()
loader.query_setup(gen)  # creates entity index on Talk.talk_id if missing

# parallel, batched ingestion
loader.ingest(gen, batchsize=BATCHSIZE, numthreads=NUMTHREADS, stats=SHOW_STATS)

print("Done. Ingestion attempted for", len(gen), "rows.")

Progress: 100%|██████████| 278/278 [00:03<00:00, 90.3items/s]

============ ApertureDB Loader Stats ============
Total time (s): 3.080139636993408
Total queries executed: 8
Avg Query time (s): 1.9688595533370972
Query time std: 0.19235428751942202
Avg Query Throughput (q/s): 4.063265958427805
Overall insertion throughput (element/s): 90.255648367735
Total inserted elements: 278
Total successful commands: 278
Done. Ingestion attempted for 278 rows.


## Sanity Check Queries

In [21]:
def run(q):
    resp, blobs = client.query(q)
    client.print_last_response()
    return resp

1) Count all talks

Confirms total inserts.

In [22]:
run([{
  "FindEntity": {
    "with_class": "Talk",
    "results": { "count": True }
  }
}])


[
    {
        "FindEntity": {
            "count": 278,
            "returned": 0,
            "status": 0
        }
    }
]


[{'FindEntity': {'count': 278, 'returned': 0, 'status': 0}}]

2) Peek newest 5 by publish date

Verifies date parsing + sorting

In [23]:
run([{
  "FindEntity": {
    "with_class": "Talk",
    "sort": { "key": "yt_published_at", "order": "descending" },
    "limit": 5,
    "results": {
      "list": ["talk_id", "talk_title", "yt_published_at", "youtube_url"]
    }
  }
}])


[
    {
        "FindEntity": {
            "entities": [
                {
                    "talk_id": "b792c10c-776b-5e2d-bb2e-3f25ea29f330",
                    "talk_title": "Building GenAI-Powered Apps: A Workshop for Software Engineers",
                    "youtube_url": "https://www.youtube.com/watch?v=tSIpREFVMXs",
                    "yt_published_at": {
                        "_date": "2025-02-13T00:00:00+00:00"
                    }
                },
                {
                    "talk_id": "36ad6014-7028-578d-800a-f969b7605df9",
                    "talk_title": "LLMidas' Touch; Safely adopting GenAI for production use-cases",
                    "youtube_url": "https://www.youtube.com/watch?v=A3KschpEU_g",
                    "yt_published_at": {
                        "_date": "2025-01-14T00:00:00+00:00"
                    }
                },
                {
                    "talk_id": "3bec210f-b106-5ff7-ab2c-3b467ff012d9",
                    "talk

[{'FindEntity': {'entities': [{'talk_id': 'b792c10c-776b-5e2d-bb2e-3f25ea29f330',
     'talk_title': 'Building GenAI-Powered Apps: A Workshop for Software Engineers',
     'youtube_url': 'https://www.youtube.com/watch?v=tSIpREFVMXs',
     'yt_published_at': {'_date': '2025-02-13T00:00:00+00:00'}},
    {'talk_id': '36ad6014-7028-578d-800a-f969b7605df9',
     'talk_title': "LLMidas' Touch; Safely adopting GenAI for production use-cases",
     'youtube_url': 'https://www.youtube.com/watch?v=A3KschpEU_g',
     'yt_published_at': {'_date': '2025-01-14T00:00:00+00:00'}},
    {'talk_id': '3bec210f-b106-5ff7-ab2c-3b467ff012d9',
     'talk_title': 'Optimizing AI/ML Workflows on Kubernetes: Advanced Techniques and Integration',
     'youtube_url': 'https://www.youtube.com/watch?v=grCvM9tkS7Q',
     'yt_published_at': {'_date': '2024-12-11T00:00:00+00:00'}},
    {'talk_id': '970e274c-873e-5059-aac0-87bb02b38e98',
     'talk_title': 'Revolutionizing the skies: Mlops case study of LATAM airlines',


3) Spot-check a specific talk_id is unique

Taking talk id one printed by #2.

In [24]:
run([{
  "FindEntity": {
    "with_class": "Talk",
    "unique": True,
    "constraints": { "talk_id": ["==", "970e274c-873e-5059-aac0-87bb02b38e98"] },
    "results": { "all_properties": True }
  }
}])


[
    {
        "FindEntity": {
            "entities": [
                {
                    "_uniqueid": "7.115.262",
                    "abstract": "This talk explores how LATAM Airlines leveraged MLOps to revolutionize their operations and achieve financial gain in the hundred of millions of dollars. By integrating machine learning models into their daily workflows and automating the deployment and management processes, LATAM Airlines was able to optimize tariffs, enhance customer experiences, and streamline maintenance operations. The talk will highlight key MLOps strategies employed, such as continuous integration and delivery of ML models, real-time data processing. Attendees will gain insights into the tangible benefits of MLOps, including cost savings, operational efficiencies, and revenue growth, showcasing how strategic ML operations can create substantial value in the airline industry.",
                    "bio": "Michael Haacke Concha is the Lead Machine Learning Engin

[{'FindEntity': {'entities': [{'_uniqueid': '7.115.262',
     'abstract': 'This talk explores how LATAM Airlines leveraged MLOps to revolutionize their operations and achieve financial gain in the hundred of millions of dollars. By integrating machine learning models into their daily workflows and automating the deployment and management processes, LATAM Airlines was able to optimize tariffs, enhance customer experiences, and streamline maintenance operations. The talk will highlight key MLOps strategies employed, such as continuous integration and delivery of ML models, real-time data processing. Attendees will gain insights into the tangible benefits of MLOps, including cost savings, operational efficiencies, and revenue growth, showcasing how strategic ML operations can create substantial value in the airline industry.',
     'bio': "Michael Haacke Concha is the Lead Machine Learning Engineer of the centralized MLOps team at LATAM Airlines. He holds both a Bachelor's and a Master’s 

4) How many talks in 2024?

Date range filter with _date.

In [25]:
run([{
  "FindEntity": {
    "with_class": "Talk",
    "constraints": {
      "yt_published_at": [
        ">=", {"_date": "2024-01-01"},
        "<=", {"_date": "2024-12-31"}
      ]
    },
    "results": { "count": True }
  }
}])


[
    {
        "FindEntity": {
            "count": 149,
            "returned": 0,
            "status": 0
        }
    }
]


[{'FindEntity': {'count': 149, 'returned': 0, 'status': 0}}]

5) Top 10 by views (descending)

Check the numeric column yt_views.

In [26]:
run([{
  "FindEntity": {
    "with_class": "Talk",
    "constraints": { "yt_views": [">", 0] },
    "sort": { "key": "yt_views", "order": "descending" },
    "limit": 10,
    "results": { "list": ["talk_title", "yt_views", "youtube_url"] }
  }
}])


[
    {
        "FindEntity": {
            "entities": [
                {
                    "talk_title": "BloombergGPT: How we built a 50 billion parameter financial language model",
                    "youtube_url": "https://www.youtube.com/watch?v=m2Scj2SO85Y",
                    "yt_views": 127705
                },
                {
                    "talk_title": "Quantifying the uncertainty in model predictions",
                    "youtube_url": "https://www.youtube.com/watch?v=-K8vDIyT3xY",
                    "yt_views": 4364
                },
                {
                    "talk_title": "Building a Measurement System for Personalization: A Bayesian Approach",
                    "youtube_url": "https://www.youtube.com/watch?v=2DZogx96aR4",
                    "yt_views": 3755
                },
                {
                    "talk_title": "Agentic AI: Unlocking Emergent Behavior in LLMs for Adaptive Workflow Automation",
                    "youtube_u

[{'FindEntity': {'entities': [{'talk_title': 'BloombergGPT: How we built a 50 billion parameter financial language model',
     'youtube_url': 'https://www.youtube.com/watch?v=m2Scj2SO85Y',
     'yt_views': 127705},
    {'talk_title': 'Quantifying the uncertainty in model predictions',
     'youtube_url': 'https://www.youtube.com/watch?v=-K8vDIyT3xY',
     'yt_views': 4364},
    {'talk_title': 'Building a Measurement System for Personalization: A Bayesian Approach',
     'youtube_url': 'https://www.youtube.com/watch?v=2DZogx96aR4',
     'yt_views': 3755},
    {'talk_title': 'Agentic AI: Unlocking Emergent Behavior in LLMs for Adaptive Workflow Automation',
     'youtube_url': 'https://www.youtube.com/watch?v=GwQi33fmexU',
     'yt_views': 2372},
    {'talk_title': 'Making ChatGPT funny with Prompt Optimization',
     'youtube_url': 'https://www.youtube.com/watch?v=lo6OOTlSS6A',
     'yt_views': 2279},
    {'talk_title': 'How many Labelled Examples do you need for a BERT-sized Model to 

6) Aggregate sanity on views

Min/Max/Average quickly surface outliers.

In [27]:
run([{
  "FindEntity": {
    "with_class": "Talk",
    "results": {
      "min": "yt_views",
      "max": "yt_views",
      "average": "yt_views"
    }
  }
}])


[
    {
        "FindEntity": {
            "_avg": {
                "yt_views": 693.205035971223
            },
            "returned": 0,
            "status": 0
        }
    }
]


[{'FindEntity': {'_avg': {'yt_views': 693.205035971223},
   'returned': 0,
   'status': 0}}]

7) Filter by speaker name (exact match)

oops - we have multiple speakers for a single talk :(

In [28]:
run([{
  "FindEntity": {
    "with_class": "Talk",
    "constraints": { "speaker_name": ["==", "Michael Haacke Concha"] },
    "results": { "list": ["talk_title", "event_name", "yt_published_at"] },
    "sort": { "key": "yt_published_at", "order": "descending" },
    "limit": 20
  }
}])


[
    {
        "FindEntity": {
            "returned": 0,
            "status": 0
        }
    }
]


[{'FindEntity': {'returned': 0, 'status': 0}}]

## speaker class addition
to cater to multiple speakers for same talk (above query issue)

In [ ]:
# same CSV
df = pd.read_csv(CSV_PATH)
df = df[df["Talk Title"].notna()].copy().reset_index(drop=True)
df = df.loc[:, ~df.columns.duplicated()].copy()

PERSON_CLASS = "Person"
EDGE_CLASS   = "TalkHasSpeaker"
TALK_CLASS   = "Talk"


In [ ]:
def split_speakers(csv_str):
    if pd.isna(csv_str) or not str(csv_str).strip():
        return []
    # split on comma, trim whitespace, drop empties
    return [s.strip() for s in str(csv_str).split(",") if s.strip()]

# We already created deterministic talk_id when ingesting Talks.
# We can simply read it back from DB. Since we didn't store talk_id in CSV, we’ll find Talk by title.
# (titles are unique.)
def talk_ref_cmd_by_title(talk_title, ref_id):
    return {
        "FindEntity": {
            "with_class": TALK_CLASS,
            "_ref": ref_id,
            "unique": True,
            "constraints": {"talk_title": ["==", talk_title]},
            "results": {"list": ["talk_id"]}  # handy for debugging
        }
    }

class SpeakerEdgesGenerator:
    """
    For each row:
      1) Find Talk by talk_title (unique)
      2) For each speaker name:
           - AddEntity Person if_not_found by name
           - AddConnection TalkHasSpeaker (src=talk, dst=person) with if_not_found
    Returns many commands per row; no blobs.
    """

    def __init__(self, dataframe: pd.DataFrame):
        self.df = dataframe

    def __len__(self): return len(self.df)

    def __getitem__(self, idx_or_slice):
        if isinstance(idx_or_slice, int):
            return self._row_to_cmds(idx_or_slice)
        if isinstance(idx_or_slice, slice):
            out = []
            start = idx_or_slice.start or 0
            stop  = idx_or_slice.stop  or len(self.df)
            step  = idx_or_slice.step  or 1
            for i in range(start, min(stop, len(self.df)), step):
                out.append(self._row_to_cmds(i))
            return out
        try:
            indices = list(idx_or_slice)
            return [self._row_to_cmds(int(i)) for i in indices]
        except Exception:
            return self._row_to_cmds(int(idx_or_slice))

    def get_indices(self):
        # index Person.name for fast upserts + optional Talk.talk_title (we use it to find Talk)
        return {
            "entity": {
                PERSON_CLASS: ["name"],
                TALK_CLASS:   ["talk_title"]
            }
        }

    def _row_to_cmds(self, i):
        r = self.df.iloc[i]
        talk_title = str(r.get("Talk Title")).strip() if pd.notna(r.get("Talk Title")) else None
        speakers   = split_speakers(r.get("Full Name"))

        if not talk_title or not speakers:
            # nothing to do for this row
            return [], []

        cmds = []
        blobs = []

        # 1) find the talk by title
        cmds.append(talk_ref_cmd_by_title(talk_title, ref_id=1))

        # 2) for each speaker: AddEntity Person (if_not_found) + AddConnection TalkHasSpeaker
        ref_counter = 2
        for name in speakers:
            # AddEntity Person
            cmds.append({
                "AddEntity": {
                    "class": PERSON_CLASS,
                    "_ref": ref_counter,
                    "if_not_found": {"name": ["==", name]},
                    "properties": {"name": name}
                }
            })
            # AddConnection TalkHasSpeaker (Talk -> Person)
            # using separate AddConnection; it supports if_not_found
            cmds.append({
                "AddConnection": {
                    "class": EDGE_CLASS,
                    "src": 1,              # Talk ref
                    "dst": ref_counter,    # Person ref
                    "if_not_found": {}     # create only if missing
                }
            })
            ref_counter += 1

        return cmds, blobs

In [ ]:
gen = SpeakerEdgesGenerator(df)

# Auto-create indices Person.name and Talk.talk_title (fast and convenient)
loader.query_setup(gen)  # uses get_indices()
# Run in parallel
loader.ingest(gen, batchsize=25, numthreads=8, stats=True)
print("Done linking speakers.")

Progress: 100%|██████████| 278/278 [00:03<00:00, 91.7items/s]

============ ApertureDB Loader Stats ============
Total time (s): 3.0314011573791504
Total queries executed: 16
Avg Query time (s): 0.9772580116987228
Query time std: 0.6979733340649226
Avg Query Throughput (q/s): 8.186169777307803
Overall insertion throughput (element/s): 91.70676712426594
Total inserted elements: 278
Total successful commands: 1024
Done linking speakers.


Find talks by a specific person (exact full name):

In [37]:
run([
  { "FindEntity": {
      "with_class": "Person",
      "_ref": 1,
      "unique": True,
      "constraints": {"name": ["==", "Michael Haacke Concha"]}
  }},
  { "FindEntity": {
      "with_class": "Talk",
      "is_connected_to": {
        "ref": 1,
        "direction": "in",
        "connection_class": "TalkHasSpeaker"
      },
      "results": {"list": ["talk_title", "speaker_name", "event_name", "yt_published_at"]},
      "sort": {"key": "yt_published_at", "order": "descending"},
      "limit": 20
  }}
])


[
    {
        "FindEntity": {
            "returned": 0,
            "status": 0
        }
    },
    {
        "FindEntity": {
            "entities": [
                {
                    "event_name": "MLOps & GenAI World 2024",
                    "speaker_name": "Michael Haacke Concha, Diego Castillo Warnken",
                    "talk_title": "Revolutionizing the skies: Mlops case study of LATAM airlines",
                    "yt_published_at": {
                        "_date": "2024-12-11T00:00:00+00:00"
                    }
                }
            ],
            "returned": 1,
            "status": 0
        }
    }
]


[{'FindEntity': {'returned': 0, 'status': 0}},
 {'FindEntity': {'entities': [{'event_name': 'MLOps & GenAI World 2024',
     'speaker_name': 'Michael Haacke Concha, Diego Castillo Warnken',
     'talk_title': 'Revolutionizing the skies: Mlops case study of LATAM airlines',
     'yt_published_at': {'_date': '2024-12-11T00:00:00+00:00'}}],
   'returned': 1,
   'status': 0}}]

List all speakers for a given talk title:

In [38]:
run([
  { "FindEntity": {
      "with_class": "Talk",
      "_ref": 1,
      "unique": True,
      "constraints": {"talk_title": ["==", "From ML Repository to ML Production Pipeline"]}
  }},
  { "FindEntity": {
      "with_class": "Person",
      "is_connected_to": {
        "ref": 1,
        "direction": "out",
        "connection_class": "TalkHasSpeaker"
      },
      "results": {"list": ["name"]},
      "sort": "name"
  }}
])


[
    {
        "FindEntity": {
            "returned": 0,
            "status": 0
        }
    },
    {
        "FindEntity": {
            "entities": [
                {
                    "name": "Dariusz Adamczyk"
                },
                {
                    "name": "Jakub Witkowski"
                }
            ],
            "returned": 2,
            "status": 0
        }
    }
]


[{'FindEntity': {'returned': 0, 'status': 0}},
 {'FindEntity': {'entities': [{'name': 'Dariusz Adamczyk'},
    {'name': 'Jakub Witkowski'}],
   'returned': 2,
   'status': 0}}]

get full schema

In [39]:
run([
  { "GetSchema": { } }
]
)

[
    {
        "GetSchema": {
            "connections": {
                "classes": {
                    "TalkHasSpeaker": {
                        "dst": "Person",
                        "matched": 373,
                        "properties": null,
                        "src": "Talk"
                    }
                },
                "returned": 1
            },
            "entities": {
                "classes": {
                    "Person": {
                        "matched": 338,
                        "properties": {
                            "name": [
                                338,
                                true,
                                "String"
                            ]
                        }
                    },
                    "Talk": {
                        "matched": 278,
                        "properties": {
                            "abstract": [
                                274,
                                f

[{'GetSchema': {'connections': {'classes': {'TalkHasSpeaker': {'dst': 'Person',
      'matched': 373,
      'properties': None,
      'src': 'Talk'}},
    'returned': 1},
   'entities': {'classes': {'Person': {'matched': 338,
      'properties': {'name': [338, True, 'String']}},
     'Talk': {'matched': 278,
      'properties': {'abstract': [274, False, 'String'],
       'bio': [269, False, 'String'],
       'category_primary': [277, False, 'String'],
       'company_name': [278, False, 'String'],
       'event_name': [242, False, 'String'],
       'industries': [157, False, 'String'],
       'job_title': [278, False, 'String'],
       'keywords_csv': [232, False, 'String'],
       'prereq_knowledge': [46, False, 'String'],
       'speaker_name': [278, False, 'String'],
       'talk_id': [278, True, 'String'],
       'talk_title': [278, True, 'String'],
       'tech_level': [238, False, 'Number'],
       'track': [261, False, 'String'],
       'unique_session_note': [108, False, 'Strin